# Simple Reinforcement Learning Loop with DQN

Although DQn is not the algoritm for the continuous action spaces, but this tutorial helps to understand how to apply RL loops to Aircontrol in general.

In [1]:
import time
from pprint import pprint
import PIL.Image as Image
from collections import deque
import base64
import os
from pprint import pprint
import numpy as np
import matplotlib.cm as cm
from io import BytesIO
from matplotlib.pyplot import  imshow
import matplotlib.pyplot as plt
import torch
import matplotlib.animation as animation
import json
import socket
import time
from tqdm import tqdm
import wandb
import pandas as pd
import random
# import dtale

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"
torch.use_deterministic_algorithms(False)
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [3]:
import airctrl
from airctrl import environment 
from airctrl import sample_generator
from dqn_agent import Agent
from airctrl.utils import unity

In [4]:
# AirControl Pypi Version
print(airctrl.__version__)

1.0.2.1


### WandB to track experiments

In [4]:
run = wandb.init(project="cesna-150",
           config={
               "batch_size": 16,
               "loss":"mse",
               "gamma": 0.9 ,
               "tau" : 0.001 ,
               "lr" : 0.0001
            },
            notes='mse loss, lidar added, tau changed, gamma chaaged , LSTM', 
            mode="disabled") # change to online to see plots on web portal

In [5]:
os.environ["WANDB_SILENT"] = "True"
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

## Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [6]:
# Launch Unity env from build
L = unity.Launch()
process = L.launch_executable("/home/supatel/Games/AirControl_2020/Build/Linux/v1.2.0-AirControl.x86_64")

In [7]:
# Define env variables
sample = sample_generator.samples()
env =  environment.Trigger()
agent = Agent(state_size=384, action_size=4, seed=0, batch_size=64,gamma=0.9,tau=0.01,lr=0.0001)
MAX_ENV_SIZE = 10000 #max distance in any direction agent can travel
MAX_EULAR = 360 # boundary for eular
MAX_LIDAR_RANGE = 500
%matplotlib inline

Now call method `.get_connected()` to get connected


In [8]:
env.get_connected()

In [9]:
def env_reset():
    output = env.reset(IsOutput=True)
    env.set_ui(ShowUIElements=True, IsActive= True)
    env.set_camera(ActiveCamera=1, IsActive=True, IsCapture=False, CaptureCamera=1, CaptureType=0,CaptureHeight=540, CaptureWidth=960)
    env.set_lidar(IsActive=False)
    return output

In [10]:
def output_to_Features(output):
    """
    Convert output from env to features
    """
    MSL = output['MSL']/MAX_ENV_SIZE
    Latitude = output['Latitude']
    Longitude = output['Longitude']
    normalizedRPM =  output['CurrentRPM']/output['MaxRPM']
    normalizedPower =  output['CurrentPower']/output['MaxPower']
    normalizedSpeed = output['CurrentSpeed']/150 # Normlizing by max speed
    pitchAngle = output['PitchAngle']
    bankAngle = output['BankAngle']
    ifCollision = output['IfCollision']
    collisionObject = output['CollisionObject']
    Reward = output["Reward"]/MAX_ENV_SIZE # Normalizing rewards
    IsGrounded = 1.0 if(output["IsGrounded"]) else 0.0
    IsFlying = 1.0 if(output["IsFlying"]) else 0.0
    IsTaxiing = 1.0 if(output["IsTaxiing"]) else 0.0
    PosXAbs = (output["PosXAbs"])/MAX_ENV_SIZE
    PosYAbs = (output["PosYAbs"])/MAX_ENV_SIZE
    PosZAbs = (output["PosZAbs"])/MAX_ENV_SIZE
    PosXRel = (output["PosXRel"])/MAX_ENV_SIZE
    PosYRel = (output["PosYRel"])/MAX_ENV_SIZE
    PosZRel = (output["PosZRel"])/MAX_ENV_SIZE
    RotXAbs = (output["RotXAbs"])/MAX_EULAR
    RotYAbs = (output["RotYAbs"])/MAX_EULAR
    RotZAbs = (output["RotZAbs"])/MAX_EULAR
    RotXRel = (output["RotXRel"])/MAX_EULAR
    RotYRel = (output["RotYRel"])/MAX_EULAR
    RotZRel = (output["RotZRel"])/MAX_EULAR
    LidarPointCloud = 1.0-np.asarray(output['LidarPointCloud'])/MAX_LIDAR_RANGE

    feature_vector = [MSL, Latitude, Longitude, normalizedRPM, normalizedPower, normalizedSpeed, pitchAngle, \
                      bankAngle, IsGrounded, IsFlying, IsTaxiing, \
                     PosXAbs, PosYAbs, PosZAbs, PosXRel, PosYRel, PosZRel,RotXAbs,RotYAbs,RotZAbs,RotXRel,RotYRel,RotZRel, ifCollision] + LidarPointCloud.tolist()

    return np.asarray(feature_vector),Reward, ifCollision,collisionObject
    


In [11]:
def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for item in my_list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
    per_freq = {}
    for key,values in freq.items():
        per_freq[key]=freq[key]/len(my_list)*100
    return per_freq

## Function to Test the model (DQN if OFF policy RL Algo.)

In [12]:
def test_run():
    # do a test run
    msl = 0
    env_output = env_reset()
    state, reward, done, collision_object = output_to_Features(env_output)
    for t in range(10000):
        if state[0]>msl:
            msl = state[0]
        _, action = agent.act(state, 0.0)
        pitch =  action[0]
        yaw = action[1]
        roll= action[2]
        stickyThrottle=action[3]
        env_output = env.step(Pitch=pitch, Yaw=yaw, Roll=roll, StickyThrottle=stickyThrottle)
        next_state, reward, done, collision_object = output_to_Features(env_output)
        state = next_state 
        wandb.log({"M/score": reward, "M/max height":msl})
        wandb.log({"M/Pitch":pitch, "M/Yaw":yaw, "M/Roll":roll, "M/StickyThrottle":stickyThrottle})
        if done:
            break

## Train

In [13]:
def dqn(n_episodes=5000, max_t=10000, eps_start=1.0, eps_end=0.05, eps_decay=0.999):
    #     global dataframe
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=1000)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        if i_episode%20==0:
            test_run()
        env_output = env_reset()
        state, reward, done, collision_object = output_to_Features(env_output)
        score = 0
        triggers = []
        msl = 0
        for t in range(max_t):
            if state[0]>msl:
                msl = state[0]
            trigger_type,action = agent.act(state, eps)
            triggers.append(trigger_type)
            pitch =  action[0]
            yaw = action[1]
            roll= action[2]
            stickyThrottle=action[3]
            env_output = env.step(Pitch=pitch, Yaw=yaw, Roll=roll, StickyThrottle=stickyThrottle)
            next_state, reward, done, collision_object = output_to_Features(env_output)
            agent.step(state, action, reward, next_state, done)
            state = next_state 
            #             dataframe = dataframe.append(env_output, ignore_index=True)
            wandb.log({"Pitch":pitch, "Yaw":yaw, "Roll":roll, "StickyThrottle":stickyThrottle})
            
            if done:
                print("🔁 Episode {0} | Counter: {1} | Collided with {2} | Score : {3} | reward : {4} | eps : {5}".format(i_episode,env_output["Counter"],collision_object, score, reward, eps))
                wandb.log({"eps": eps, "score": score, "score per iteration" : score/(t+1e-6), "episode size":t,"max height":msl})
                wandb.log(CountFrequency(triggers))
                break
            score += reward
        scores_window.append(score)       # save most recent score
        scores.append(score)    # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
            
    return scores

In [17]:
scores = dqn()

Error: You must call wandb.init() before wandb.log()

In [15]:
wandb.finish()

In [ ]:
process.terminate()